In [213]:
import random
import string
import numpy as np
import pandas as pd
from tabulate import tabulate

### Config

In [214]:
DISH_COUNT = 300
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1200
CALORIES_PER_DAY = 2000
DAYS_PER_MENU = 7
DISHES_PER_DAY = 3

PENALTY = 400
POPULATION_SIZE = 20
MATING_POPULATION_SIZE = 10
ITERATIONS = 3000

REAL_DATA = True

#### Helper functions

In [215]:
def visualize_menu(menu):
    print("Score: " + str(score_menu(menu)))
    headers = ["Day", "Calories", "Dishes"]
    calories = count_calories_per_day(menu)
    table = []
    for day in range(DAYS_PER_MENU):
        table.append([day, calories[day], menu[day]])
    print(tabulate(table, headers, tablefmt="fancy_grid"))

#### Test data

In [216]:
def fake_data():
    dishes = {}
    for _ in range(DISH_COUNT):
        name = ''.join(random.choices(string.ascii_lowercase, k=6))
        dishes[name] = random.randint(MIN_CALORIES_PER_DISH, MAX_CALORIES_PER_DISH)
    dishes = list(dishes.items())
    return dishes

#### Real data

In [217]:
def real_data():
    dishes = {}
    data = pd.read_csv('data.csv', delimiter=',', encoding='utf-8', low_memory=False)
    df = pd.DataFrame(data, columns=['name', 'Food Group', "calories_portion"])

    while len(dishes) != DISH_COUNT:
        random_choice = df.sample()
        if MIN_CALORIES_PER_DISH < random_choice.calories_portion.values[0] < MAX_CALORIES_PER_DISH:
            dishes[random_choice.name.values[0]] = random_choice.calories_portion.values[0]
    dishes = list(dishes.items())
    return dishes

In [218]:
dishes = {}
if REAL_DATA:
    dishes = real_data()
else:
    dishes = fake_data()
print(*dishes[:7], sep='\n')

('Spaghetti Sauce With Meat And Added Vegetables', 218.4)
('Eggnog Lowfat / Light', 189.44)
('Double Bacon Cheeseburger 2 Medium Patties With Condiments On Bun From Fast Food / Restaurant', 907.85)
('Cod Baked Or Broiled Made Without Fat', 147.9)
('Whiting Cooked Ns As To Cooking Method', 249.73)
('Lamb Domestic Shoulder Arm Separable Lean And Fat Trimmed To 1/4 Inch Fat Choice Cooked Roasted', 237.15)
('Beans Yellow Mature Seeds Raw', 676.2)


#### Sample random menu

In [219]:
def create_random_menu():
    menu = np.ndarray((7, 3), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [220]:
def count_calories_per_day(menu):
    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]

    return calories


def count_attr_per_day(menu):
    names = np.reshape(menu, 21)
    unique, counts = np.unique(names, return_counts=True)
    names_dict = dict(zip(unique, counts))

    attr = []
    for day in range(DAYS_PER_MENU):
        attr.append(0)
        for dish_index in range(DISHES_PER_DAY):
            attr[-1] += (names_dict[menu[day, dish_index]] - 1) * PENALTY

    return attr


def score_menu(menu):
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    penalty = [attr ** 2 for attr in count_attr_per_day(menu)]
    squares = [calories ** 2 for calories in differences]

    return sum(squares + penalty)


def check_duplications(menu):
    names = np.reshape(menu, 21)
    unique, counts = np.unique(names, return_counts=True)
    is_repeating = dict(zip(unique, counts))

    for elem in is_repeating:
        if is_repeating[elem] > 2:
            print("duplicants detected")
            return
    print("No duplicants")

In [221]:
sample_menu = create_random_menu()
visualize_menu(sample_menu)

Score: 9544128.046775999
╒═══════╤════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                                                   │
╞═══════╪════════════╪══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    821.84  │ [('Salmon Coated Fried Made With Oil', 430.1)                                                                                            │
│       │            │  ('Eggnog Lowfat / Light', 189.44)                                                                                                       │
│       │            │  ('Veal Leg (Top Round) Separable Lean And Fat Cooked Pan-Fried Breaded', 202.3)]                                                        │
├──

### Algorithm

In [222]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child


def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU - 1), random.randint(0, DISHES_PER_DAY - 1)] = random.choice(dishes)
    return menu


def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU - 1)
    second_day_index = random.randint(0, DAYS_PER_MENU - 1)
    dish_index = random.randint(0, DISHES_PER_DAY - 1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[
        first_day_index, dish_index]
    return menu

In [223]:
def initial_population():
    population = []
    for _ in range(POPULATION_SIZE):
        population.append(create_random_menu())
    return population

In [230]:
def genetic_algorithm():
    population = initial_population()
    for i in range(ITERATIONS):
        population.sort(key=score_menu)
        mating_population = population[:MATING_POPULATION_SIZE]

        offspring_population = []
        for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
            offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
            if random.random() < 0.1:
                offspring_population[-1] = random_mutate(offspring_population[-1])
            if random.random() < 0.1:
                offspring_population[-1] = swap_mutate(offspring_population[-1])

        population = mating_population + offspring_population

    population.sort(key=score_menu)
    best_individual, worst_individual = population[0], population[-1]

    return best_individual, worst_individual

### Example

In [231]:
best_individual, worst_individual = genetic_algorithm()
visualize_menu(best_individual)
check_duplications(best_individual)

Score: 54525.16129999991
╒═══════╤════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                                                           │
╞═══════╪════════════╪══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    1886.86 │ [('Beans Yellow Mature Seeds Raw', 676.2) ('Crisp Blueberry', 629.76)                                                                            │
│       │            │  ('Cornmeal Degermed Enriched White', 580.9)]                                                                                                    │
├───────┼────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────

### Experiments

In [234]:
PENALTY = 400
POPULATION_SIZE = 20
MATING_POPULATION_SIZE = 10
ITERATIONS = 3000
best_individual, worst_individual = genetic_algorithm()
print(score_menu(best_individual), score_menu(worst_individual))

48380.20749999996 439263.25719999993
50922.72989999996 425610.8519
